In [45]:
import json
import pandas as pd 
import numpy as np
from datetime import date
import pytz

In [46]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [47]:
with open("trip_update.json", "r") as read_file:
    data = json.load(read_file)

In [48]:
entities = data['entity']
entity_id = []
entity_timestamp = []
entity_trip_id = []
entity_route_id = []
entity_stop_id = []
entity_scheduleRelationship = []

entity_arrival_time = []
entity_arrival_delay = []

entity_departure_time = []
entity_departure_delay = []

entity_stop_time_update_scheduleRelationship = []


In [49]:
def get_data():
    for entity in entities:
        entityTimestamp = entity['tripUpdate']['timestamp']
        entityTripId = entity['tripUpdate']['trip']['tripId']
        entityRouteId = entity['tripUpdate']['trip']['routeId']

        # if the entity has the 'scheduleRelationship' key
        if 'scheduleRelationship' in entity['tripUpdate']['trip']:
            entityScheduleRelationship = entity['tripUpdate']['trip']['scheduleRelationship']
        else:
            entityScheduleRelationship = ''

        # if the entity has the 'stopTimeUpdate' key
        if 'stopTimeUpdate' in entity['tripUpdate']:
            entity_stop_time_update = entity['tripUpdate']['stopTimeUpdate']

            for stop_time_update in entity_stop_time_update:
                entity_timestamp.append(entityTimestamp)
                entity_trip_id.append(entityTripId)
                entity_scheduleRelationship.append(entityScheduleRelationship)
                entity_route_id.append(entityRouteId)
                entity_stop_id.append(stop_time_update['stopId'])

                # if the entity has the 'arrival' key
                if 'arrival' in stop_time_update:
                    if ('time' in stop_time_update['arrival']) & ('delay' not in stop_time_update['arrival']):
                        entity_arrival_time.append(stop_time_update['arrival']['time'])
                        entity_arrival_delay.append(0)

                    elif ('delay' in stop_time_update['arrival']) & ('time' not in stop_time_update['arrival']):
                        entity_arrival_delay.append(stop_time_update['arrival']['delay'])
                        entity_arrival_time.append(None)
                    else:
                        entity_arrival_time.append(stop_time_update['arrival']['time'])
                        entity_arrival_delay.append(stop_time_update['arrival']['delay'])
                else:
                    entity_arrival_delay.append(0)
                    entity_arrival_time.append(None)

                # if the entity has the 'departure' key
                if 'departure' in stop_time_update:
                    if ('time' in stop_time_update['departure']) & ('delay' not in stop_time_update['departure']):
                        entity_departure_time.append(stop_time_update['departure']['time'])
                        entity_departure_delay.append(0)

                    elif ('delay' in stop_time_update['departure']) & ('time' not in stop_time_update['departure']):
                        entity_departure_delay.append(stop_time_update['departure']['delay'])
                        entity_departure_time.append(None)

                    else:
                        entity_departure_time.append(stop_time_update['departure']['time'])
                        entity_departure_delay.append(stop_time_update['departure']['delay'])

                else:
                    entity_departure_delay.append(0)
                    entity_departure_time.append(None)

                # if the entity has the 'scheduleRelationship' key
                if 'scheduleRelationship' in stop_time_update:
                    entity_stop_time_update_scheduleRelationship.append(stop_time_update['scheduleRelationship'])
                else:
                    entity_stop_time_update_scheduleRelationship.append('')
        else:
            entity_arrival_delay.append(0)
            entity_arrival_time.append(None)
            entity_departure_delay.append(0)
            entity_departure_time.append(None)
            entity_stop_id.append('')
            entity_stop_time_update_scheduleRelationship.append('')
            entity_timestamp.append(entityTimestamp)
            entity_trip_id.append(entityTripId)
            entity_scheduleRelationship.append(entityScheduleRelationship)
            entity_route_id.append(entityRouteId)

In [50]:
get_data()

In [51]:
df = pd.DataFrame(columns = ['timestamp', 'trip_id','route_id','schedule_relationship',
                            'stop_id','departure_delay','departure_time','arrival_delay',
                            'arrival_time','update_scheduleRelationship'])

df['timestamp'] = pd.Series(entity_timestamp)
df['trip_id'] = pd.Series(entity_trip_id)
df['route_id'] = pd.Series(entity_route_id)
df['schedule_relationship'] = pd.Series(entity_scheduleRelationship)
df['stop_id'] = pd.Series(entity_stop_id)
df['departure_delay'] = pd.Series(entity_departure_delay)
df['departure_time'] = pd.Series(entity_departure_time)
df['arrival_delay'] = pd.Series(entity_arrival_delay)
df['arrival_time'] = pd.Series(entity_arrival_time)
df['update_scheduleRelationship'] = pd.Series(entity_stop_time_update_scheduleRelationship)

In [52]:
# Drop the rows of Nsw Trains
df = df[~(df["route_id"].isin(['BMT_1', 'BMT_2', 'CCN_1a', 'CCN_1b', 'CCN_1c',
                               'CCN_2a', 'CCN_2b', 'CTY_NC1', 'CTY_NC1a',
                               'CTY_NC2', 'CTY_NW1a', 'CTY_NW1b', 'CTY_NW1c',
                               'CTY_NW1d', 'CTY_NW2a', 'CTY_NW2b', 'CTY_S1a',
                               'CTY_S1b', 'CTY_S1c', 'CTY_S1d', 'CTY_S1e',
                               'CTY_S1f', 'CTY_S1g', 'CTY_S1h', 'CTY_S1i',
                               'CTY_S2a', 'CTY_S2b', 'CTY_S2c', 'CTY_S2d',
                               'CTY_S2e', 'CTY_S2f', 'CTY_S2g', 'CTY_S2h',
                               'CTY_S2i', 'CTY_W1a', 'CTY_W1b', 'CTY_W2a',
                               'CTY_W2b', 'HUN_1a', 'HUN_1b', 'HUN_2a',
                               'HUN_2b', 'SCO_1a', 'SCO_1b', 'SCO_2a',
                               'SCO_2b', 'SHL_1a', 'SHL_1b', 'SHL_1c',
                               'SHL_1d', 'SHL_1e', 'SHL_2a', 'SHL_2b', 'SHL_2c',
                               'SHL_2d', 'SHL_2e', 'RTTA_DEF', 'RTTA_REV']))]

In [53]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.timestamp = df.timestamp + pd.Timedelta('10:00:00')
df.timestamp = df.timestamp.dt.strftime('%Y%m%d %H:%M:%S')


df.departure_time = pd.to_datetime(df['departure_time'], unit='s')
df.departure_time = df.departure_time + pd.Timedelta('10:00:00')
df.departure_time = df.departure_time.dt.strftime('%Y%m%d %H:%M:%S')

df.arrival_time = pd.to_datetime(df['arrival_time'], unit='s')
df.arrival_time = df.arrival_time + pd.Timedelta('10:00:00')
df.arrival_time = df.arrival_time.dt.strftime('%Y%m%d %H:%M:%S')


In [54]:
# df['arrival_time'] = df['arrival_time'].where(df['arrival_time'].notnull(), None)
# df.replace({pd.NaT: None},inplace=True)
df = df.replace({'NaT': None})

In [55]:
df.departure_time.unique()

array([nan, '20200414 09:10:31', '20200414 09:14:11', '20200414 09:16:00',
       '20200414 09:19:09', '20200414 09:21:20', '20200414 09:23:40',
       '20200414 09:27:21', '20200414 09:31:00', '20200414 09:35:10',
       '20200414 09:36:41', '20200414 09:39:11', '20200414 09:40:50',
       '20200414 09:42:20', '20200414 09:43:50', '20200414 09:46:11',
       '20200414 09:48:30', '20200414 09:50:30', '20200414 09:52:49',
       '20200414 13:37:36', '20200414 13:40:54', '20200414 13:44:00',
       '20200414 13:46:07', '20200414 13:49:04', '20200414 13:50:54',
       '20200414 13:54:17', '20200414 13:58:30', '20200414 14:01:36',
       '20200414 14:03:54', '20200414 14:06:24', '20200414 14:08:54',
       '20200414 14:11:24', '20200414 14:13:54', '20200414 14:16:06',
       '20200414 14:18:24', '20200414 14:21:12', '20200414 14:23:24',
       '20200414 14:25:54', '20200414 14:28:24'], dtype=object)

In [56]:
df['date'] = pd.to_datetime(df['timestamp']).dt.date
df['time'] = pd.to_datetime(df['timestamp']).dt.time
df.drop('timestamp',axis=1,inplace=True)

In [57]:
# rearrage the order of columns
df = df[['date','time','trip_id','route_id','schedule_relationship','stop_id','arrival_time',
        'arrival_delay','departure_time','departure_delay','update_scheduleRelationship']]
df.reset_index(drop=True,inplace=True)

In [59]:
df.head()

,date,time,trip_id,route_id,schedule_relationship,stop_id,arrival_time,arrival_delay,departure_time,departure_delay,update_scheduleRelationship
0,2020-04-14,01:34:00,20XZ.1649.103.198.B.8.61078906,BNK_1d,SCHEDULED,,NaN,0,NaN,0,
1,2020-04-14,01:45:31,93-S.1649.103.4.A.8.61080230,BNK_2b,SCHEDULED,,NaN,0,NaN,0,
2,2020-04-14,00:14:46,120W.1649.103.4.A.8.61080516,NTH_1a,SCHEDULED,,NaN,0,NaN,0,
3,2020-04-14,01:24:39,127W.1649.103.4.A.8.61080557,NSN_1a,SCHEDULED,2077302,NaN,0,NaN,0,SCHEDULED
4,2020-04-14,00:53:48,10XZ.1649.103.4.B.8.61081877,BNK_2a,SCHEDULED,,NaN,0,NaN,0,


In [62]:
# Get the timetable current date
file_date = date.today().strftime("%Y%m%d")

In [44]:
df.to_csv('trip_update_' + file_date + '.csv',index=False)